In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install autogluon.tabular[all]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.3/218.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━

In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [5]:
train = TabularDataset('/content/drive/MyDrive/KT_Aivle_School/Data/Contest/AISPARK_4th/train_data.csv')
test = TabularDataset('/content/drive/MyDrive/KT_Aivle_School/Data/Contest/AISPARK_4th/test_data.csv')

In [6]:
hp_dict = {0: 30,
           1: 20,
           2: 10,
           3: 50,
           4: 20,
           5: 30,
           6: 30,
           7: 30}

train['HP'] = train['type'].map(hp_dict)
test['HP'] = test['type'].map(hp_dict)

In [7]:
train.drop('type', axis=1, inplace=True)
test.drop('type', axis=1, inplace=True)

In [8]:
# from sklearn.model_selection import train_test_split

# train, valid = train_test_split(train, test_size=0.2, random_state=42, stratify=train['HP'])

In [9]:
label = 'HP'
time_limit = 120
predictor = TabularPredictor(label=label).fit(train, time_limit=time_limit)

No path specified. Models will be saved in: "AutogluonModels/ag-20230417_164837/"
Beginning AutoGluon training ... Time limit = 120s
AutoGluon will save models to "AutogluonModels/ag-20230417_164837/"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Dec 10 16:00:40 UTC 2022
Train Data Rows:    2463
Train Data Columns: 7
Label Column: HP
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	4 unique label values:  [30, 20, 10, 50]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 4
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12557.84 M

In [10]:
result = predictor.predict(test)

In [11]:
result

0       30
1       30
2       30
3       30
4       30
        ..
7384    30
7385    30
7386    30
7387    30
7388    30
Name: HP, Length: 7389, dtype: int64

In [12]:
y = pd.DataFrame()
y['HP'] = test['HP']
y['pred'] = result

In [13]:
y['label'] = np.nan
for i in range(y.shape[0]):
    if y.iat[i, 0] == y.iat[i, 1]:
        y.iat[i, 2] = 1
    else:
        y.iat[i, 2] = 0


In [14]:
y['label'].isnull().sum()

0

In [15]:
submit = pd.read_csv('/content/drive/MyDrive/KT_Aivle_School/Data/Contest/AISPARK_4th/answer_sample.csv')
submit

,type,label
0,0,-1
1,0,-1
2,0,-1
3,0,-1
4,0,-1
...,...,...
7384,7,-1
7385,7,-1
7386,7,-1
7387,7,-1


In [16]:
submit['label'] = y['label'].astype('int64')
submit

,type,label
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
7384,7,1
7385,7,1
7386,7,1
7387,7,1


In [17]:
import os
submit.to_csv(os.path.join('/content/drive/MyDrive/KT_Aivle_School/Data/Contest/AISPARK_4th', 'submit.csv'))

In [18]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.0 MB/s eta 0:00:00


In [19]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.8/105.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
 

In [22]:
from pycaret.classification import *

In [23]:
exp_class = setup(data=train, target='HP', session_id=42, normalize=True)

,Description,Value
0,Session id,42
1,Target,HP
2,Target type,Multiclass
3,Target mapping,"10: 0, 20: 1, 30: 2, 50: 3"
4,Original data shape,"(2463, 8)"
5,Transformed data shape,"(2463, 8)"
6,Transformed train set shape,"(1724, 8)"
7,Transformed test set shape,"(739, 8)"
8,Numeric features,7
9,Preprocess,True


In [24]:
best = compare_models(include=['lightgbm', 'rf', 'mlp', 'knn'], sort='F1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
mlp,MLP Classifier,0.8718,0.9121,0.8718,0.9002,0.8600,0.8054,0.8278,1.8190
knn,K Neighbors Classifier,0.8475,0.8965,0.8475,0.8521,0.8398,0.7712,0.7807,0.1090
lightgbm,Light Gradient Boosting Machine,0.8376,0.9072,0.8376,0.8379,0.8321,0.7577,0.7633,1.6790
rf,Random Forest Classifier,0.8080,0.9001,0.8080,0.8076,0.8068,0.7176,0.7187,0.8210


Processing:   0%|          | 0/21 [00:00<?, ?it/s]

In [25]:
tuned_best = tune_model(best, optimize='f1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8671,0.8986,0.8671,0.8970,0.8540,0.7973,0.8212
1,0.8786,0.9150,0.8786,0.9044,0.8688,0.8165,0.8364
2,0.8786,0.9336,0.8786,0.9044,0.8688,0.8165,0.8364
3,0.8671,0.9071,0.8671,0.8973,0.8546,0.7985,0.8221
4,0.8837,0.9072,0.8837,0.9075,0.8746,0.8241,0.8425
5,0.8547,0.8978,0.8547,0.8899,0.8384,0.7783,0.8065
6,0.8721,0.9401,0.8721,0.9002,0.8605,0.8059,0.8279
7,0.8488,0.8604,0.8488,0.8866,0.8307,0.7691,0.7994
8,0.8895,0.9300,0.8895,0.9112,0.8815,0.8331,0.8498


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [27]:
save_model(tuned_best, '/content/drive/MyDrive/KT_Aivle_School/Data/6th_mini_project/best_lgbm')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['air_inflow', 'air_end_temp',
                                              'out_pressure', 'motor_current',
                                              'motor_rpm', 'motor_temp',
                                              'motor_vibe'],
                                     transformer=SimpleImputer(add_i...
                                batch_size='auto', beta_1=0.9, beta_2=0.999,
                                early_stopping=False, epsilon=1e-08,
                                hidden_layer_sizes=(100,),
                                learning_rate='constant',
                                learning_rate